In [2]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

def load_images_and_labels(folder_path, image_width, image_height):
    images = []
    labels = []
    for category in os.listdir(folder_path):
        category_path = os.path.join(folder_path, category)
        if os.path.isdir(category_path):  
            for image_name in os.listdir(category_path):
                image_path = os.path.join(category_path, image_name)
                if os.path.isfile(image_path):  
                    image = cv2.imread(image_path)
                    if image is not None:
                        image = cv2.resize(image, (image_width, image_height))
                        images.append(image)
                        labels.append(category)
                    else:
                        print(f"Warning: {image_path} could not be read.")
                else:
                    print(f"Warning: {image_path} is not a file.")
        else:
            print(f"Warning: {category_path} is not a directory.")
    return np.array(images), np.array(labels)

image_width = 100
image_height = 100

train_folder_path = r"D://Dataset_t//train"
X_train, y_train = load_images_and_labels(train_folder_path, image_width, image_height)

print(f"Loaded {len(X_train)} training images.")
print(f"Loaded {len(y_train)} training labels.")

test_folder_path = r"D://Dataset_t//test"
X_test, y_test = load_images_and_labels(test_folder_path, image_width, image_height)

print(f"Loaded {len(X_test)} test images.")
print(f"Loaded {len(y_test)} test labels.")

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_train_one_hot = to_categorical(y_train_encoded)

y_test_encoded = label_encoder.transform(y_test)
y_test_one_hot = to_categorical(y_test_encoded)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2, random_state=42)

# Normalize images
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

#  CNN 
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation='relu')) 
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(len(label_encoder.classes_), activation='softmax'))

optimizer = Adam(learning_rate=0.0001)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_val, y_val),
                    epochs=50,
                    callbacks=[early_stopping])


test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

model.save("my_model.h5")


Loaded 380 training images.
Loaded 380 training labels.
Loaded 295 test images.
Loaded 295 test labels.
Epoch 1/50
10/10 [==============================] - 6s 455ms/step - loss: 1.1612 - accuracy: 0.5757 - val_loss: 0.5895 - val_accuracy: 0.9211
Epoch 2/50
10/10 [==============================] - 4s 409ms/step - loss: 0.9443 - accuracy: 0.5395 - val_loss: 0.6468 - val_accuracy: 0.8684
Epoch 3/50
10/10 [==============================] - 4s 419ms/step - loss: 1.1421 - accuracy: 0.5559 - val_loss: 0.7277 - val_accuracy: 0.2895
Epoch 4/50
10/10 [==============================] - 5s 473ms/step - loss: 0.9600 - accuracy: 0.6053 - val_loss: 0.7695 - val_accuracy: 0.1974
Epoch 5/50
10/10 [==============================] - 4s 401ms/step - loss: 1.0111 - accuracy: 0.5428 - val_loss: 0.8293 - val_accuracy: 0.1053
Epoch 6/50
10/10 [==============================] - 1s 77ms/step - loss: 0.5761 - accuracy: 0.9525
Test Loss: 0.576079249382019
Test Accuracy: 0.9525423645973206


In [3]:
import cv2
import mediapipe as mp
from keras.models import load_model
import numpy as np

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

model_path = "my_model.h5"
exercise_label = "barbell biceps curl"

model = load_model(model_path)

cap = cv2.VideoCapture(0)

correct_counter = 0  

def classify_exercise(image, pose):
    try:
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks is None:
            return "Detection Error"

        landmarks = results.pose_landmarks.landmark

        # Extract  points
        l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

        r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

        # Calculating angles for biceps curl
        def calculate_angle(a, b, c):
            a = np.array(a)  
            b = np.array(b)  
            c = np.array(c)  

            radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
            angle = np.abs(radians * 180.0 / np.pi)
            if angle > 180.0:
                angle = 360.0 - angle
            return angle

        l_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)
        r_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)

        # Formatting data for the model
        data = np.array([[l_angle, r_angle]])

        # Classifying the exercise using the model
        prediction = model.predict(data)
        exercise = exercise_label if prediction[0][0] > 0.5 else "Not " + exercise_label

        return exercise

    except:
        return "Detection Error"

def draw_connections(frame, results):
    if results.pose_landmarks is not None:
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
        )

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (1300, 700))
        cv2.putText(frame, "Press 1 for Barbell Biceps Curl Exercise", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, "Press q to exit", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv2.imshow('Mediapipe Feed', frame)

        key = cv2.waitKey(1)
        if key & 0xFF == ord('1'):
            cv2.destroyAllWindows()
            correct_counter = 0  
            while cap.isOpened():
                ret, frame = cap.read()
                frame = cv2.resize(frame, (1300, 700))
                exercise_result = classify_exercise(frame, pose)
                result_message = "Correct" if exercise_result == exercise_label else "Incorrect"
                if result_message == "Correct":
                    correct_counter += 1  
                cv2.putText(frame, result_message, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                draw_connections(frame, pose.process(frame))
                cv2.putText(frame, f"Correct Count: {correct_counter}", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  # Display correct count
                cv2.imshow("Exercise", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    cv2.destroyAllWindows()
                    break

        elif key & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
